In [ ]:
import os
import sys
import glob
os.environ["HOME"]

In [ ]:
import pandas as pd
import numpy as np
from functools import reduce
import bioinfokit
from bioinfokit import analys, visuz
from bioinfokit.analys import norm, get_data

In [ ]:
folder_count = '_starReadsPerGene.out.tab_FILE_PATH'

count_files = glob.glob(os.path.join(folder_count, "*_starReadsPerGene.out.tab"))
count_tableset = []
for filename in count_files:
    df_count = pd.read_csv(filename, sep = '\t')
    count_tableset.append(df_count)

In [ ]:
new_count_f = []
for valname in count_files:
    spliced_name = valname.split('/')[-1].split('.')[0]
    new_count_f.append(spliced_name)
dict_count_df = dict(zip(new_count_f, count_tableset))

In [ ]:
#Remove the unstranded reads and sense or anti-sense read according to the data strandedness.
for key, val in dict_count_df.items():
    val.drop(['unstranded', str(key)], axis = 1, inplace = True)

In [ ]:
#Assign the variables with each sample's dataframe : each name of merged columns = dict_count_df[str(key)]
well_1 = dict_count_df['well_1']
well_2 = dict_count_df['well_2']
well_3 = dict_count_df['well_3']
chip_1 = dict_count_df['chip_1']
chip_2 = dict_count_df['chip_2']
chip_3 = dict_count_df['chip_3']

In [ ]:
#Merge the sample's read count in one dataframe
merge_sample = [well_1, well_2, well_3, chip_1, chip_2, chip_3]
merge_count = reduce(lambda left, right : pd.merge(left, right, on = ['gene_ID'], how = 'outer'), merge_sample)
merge_count = merge_count.fillna(0)
merge_count.columns = ["gene_ID", "Tw1", "Tw2", "Tw3", "C1", "C2", "C3"]

In [ ]:
#Remove the outlier sequences (if 0 appears in more than half of the sample columns, it is designated as an outlier and filtered out.)
merge_count['zero_Count'] = (merge_count[["Tw1", "Tw2", "Tw3", "C1", "C2", "C3"]] == 0).sum(axis=1)
merge_count = merge_count[merge_count['zero_Count'] < 4]
merge_count = merge_count.drop(['zero_Count'], axis = 1)

In [ ]:
#Reference file to match the gene id to the gene name is required.
#mus_musculus_geneID_genename_v105.txt file is provided in '/process_data/'.

folder_ref = 'REFERENCE_FILE_PATH'
gene_matching = pd.read_csv('{}/mus_musculus_geneID_genename_v105.txt'.format(folder_ref), sep = ',')
gene_id = gene_matching["gene_ID"].tolist()
gene = gene_matching["gene"].tolist()
new_gene_id = []
for i in gene_id:
    new_gene = i.split(".")[0]
    new_gene_id.append(new_gene)
    
dict_gene = dict(zip(new_gene_id, gene))
df_gene = pd.DataFrame.from_dict(data = dict_gene, orient = 'index')
df_gene = df_gene.reset_index()
df_gene.columns = ["gene_ID", "gene"]

In [ ]:
#Process the dataframe to merge the counts of duplicates

merge_geneid = pd.merge(df_gene, merge_count, on = 'gene_ID', how = 'inner')
merge_geneid = merge_geneid.dropna()
merge_geneid2 = merge_geneid.drop(["gene_ID"], axis = 1)
merge_gene_sum = merge_geneid2.groupby(['gene']).sum()
merge_gene_sum = merge_gene_sum.reset_index()
genecount_filter = merge_gene_sum[["gene", "Tw1", "Tw2", "Tw3", "C1", "C2", "C3"]]
genecount_filter = genecount_filter.drop_duplicates().groupby('gene', sort = False, as_index = False).sum()

In [ ]:
#Make the table with gene CPM instead of the count using the bioinfokit toolkit.

sum_forCPM = genecount_filter.groupby(['gene']).sum()
df_forCPM = sum_forCPM.dropna()
nm = norm()
nm.cpm(df = df_forCPM)
CPM_findf = nm.cpm_norm
CPM_findf.head()